# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [23]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [24]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append to full_data_rows_list     
        for line in csvreader:
            #print(line)
            #Each line is a list of strings and full_data_rows_list is a list of lists
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating one event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables. This csv file has data from all the source csv files
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [26]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [27]:
# making a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, creating a session object from cluster object
session = cluster.connect()

#### Create Keyspace

In [28]:
# Creating a Keyspace using the method session.execute
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS kkd 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [29]:
# Setting KEYSPACE to 'kkd' for the session
try:
    session.set_keyspace('kkd')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [30]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
# the combination sessionID, iteminSession is unique and is being chosen as primary key as per query requirement
#The table columns have been decided based on primary key and query requirement

query = "DROP TABLE IF EXISTS music_library1 "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS music_library1 "
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [31]:
#Reading from event_datafile_new.csv and writing into the table music_library1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Each line is a list of strings
        ## Assigning the INSERT statements into the `query` variable
        #print(line) 
        query = "INSERT INTO music_library1 (sessionid, iteminsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [32]:
# Verifying the data was correctly entered into the table
query = "SELECT artist, song, length FROM music_library1 WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print("artist:", row.artist, "\tsong:", row.song, "\tlength:", row.length);

artist: Faithless 	song: Music Matters (Mark Knight Dub) 	length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [33]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# (userID, sessionID, iteminSession) is being chosen as primary key as per query requirement, uniqueness and sort requirement
#The table columns have been decided based on primary key and query requirement

query = "DROP TABLE IF EXISTS music_library2 "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS music_library2 "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, user_first text, user_last text, \
                    PRIMARY KEY (userid, sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# Source data from event_datafile_new.csv and populate music_library2 table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Each line is a list of strings
        ## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_library2 (userid, sessionid, iteminsession, artist, song, user_first, user_last)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    
            
## Adding in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, user_first, user_last, iteminsession FROM music_library2 WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print("Artist:", row.artist, "\tSong:", row.song, "\tUser Name:", row.user_first, row.user_last, \
          "\titemInsession:", row.iteminsession);

Artist: Down To The Bone 	Song: Keep On Keepin' On 	User Name: Sylvie Cruz 	itemInsession: 0
Artist: Three Drives 	Song: Greece 2000 	User Name: Sylvie Cruz 	itemInsession: 1
Artist: Sebastien Tellier 	Song: Kilometer 	User Name: Sylvie Cruz 	itemInsession: 2
Artist: Lonnie Gordon 	Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 	User Name: Sylvie Cruz 	itemInsession: 3


In [34]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# song, user_first, user_last is being chosen as primary key as per query requirement and uniqueness requirement
# It is possible that an user can listen to the same song multipe times (with different sessionID, iteminSession) but 
# that is OK for Query 3 requirement
#The table columns have been decided based on primary key and query requirement
query = "DROP TABLE IF EXISTS music_library3 "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS music_library3 "
query = query + "(song text, user_first text, user_last text, \
                    PRIMARY KEY (song, user_first, user_last))"

try:
    session.execute(query)
except Exception as e:
    print(e)

# Source data from event_datafile_new.csv and populate music_library3 table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Each line is a list of strings
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library3 (song, user_first, user_last)"
        query = query + "VALUES (%s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (line[9], line[1], line[4]))
                    
          
# Adding in the SELECT statement to verify the data was entered into the table 
query = "SELECT * FROM music_library3 WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print("User Name:", row.user_first, row.user_last);

                    

User Name: Jacqueline Lynch
User Name: Sara Johnson
User Name: Tegan Levine


### Drop the tables before closing out the sessions

In [35]:
## Drop the table before closing out the sessions

In [36]:
query = "DROP TABLE IF EXISTS music_library1 "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_library2 "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_library3 "
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()